# Библиотека NumPy

Прежде, чем перейти к линейной регрессии разберем несколько функций из библиотеки NumPy, которые нам очень пригодятся на уроке.
Как мы помним библиотека NumPy позволяет делать преобразования с матрцами.

Также напомню про шпаргалку из первого урока: https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf


А если хочется погрузится в него поглубже, есть гитхаб с упражнениями
https://github.com/rougier/numpy-100/blob/master/100_Numpy_exercises_with_solutions.md

Импортируем библиотеку

In [1]:
import numpy as np

**Создадим массив, состоящий из случайных элементов**

In [2]:
Z = np.random.random((3,3))
print(Z)

[[0.95899845 0.52288739 0.68520693]
 [0.09100422 0.71860994 0.10296366]
 [0.14398612 0.03480088 0.73654782]]


 **Мы видим, что он состоит из трех рядов и из трех колонок, мы часто будем работать с такого вида данными, где колонки соотносятся с фичами**

In [3]:
Z[:, :]

array([[0.95899845, 0.52288739, 0.68520693],
       [0.09100422, 0.71860994, 0.10296366],
       [0.14398612, 0.03480088, 0.73654782]])

Первое двоеточие отвечает за то, к каким строкам мы хотим обращаться, а второе к каким столбцам.
Если они написаны, как в примере выше, это означает, что мы хотим видеть все столбцы и все колонки.

Попробуем обратиться только  к некоторым строкам или только к некоторым столбцам, это делается слудующими командами

**1) Выведем все колонки кроме последней**

In [4]:
Z[:, :-1]

array([[0.95899845, 0.52288739],
       [0.09100422, 0.71860994],
       [0.14398612, 0.03480088]])

**2) Выведем все строки кроме последней**

In [5]:
Z[:-1, :]

array([[0.95899845, 0.52288739, 0.68520693],
       [0.09100422, 0.71860994, 0.10296366]])

**3) Выведем  первую колонку**

In [6]:
Z[:, 0]

array([0.95899845, 0.09100422, 0.14398612])

# Линейная регрессия

Напоним, что такое линейная регрессия:

Также есть ряд предположений:
* Предполагается, что связь между переменными линейная.
* Предполагается, что величина предсказанные значения минус наблюдаемые имеет нормальное распределение
* Предполагается, что величина предсказанные значения минус наблюдаемые имеет постоянную дисперсию

Мы изучили теорию, как оптимизировать функционал качества и находить минимум.\
Теперь можно попробовать применить наши знания на реальных данных.\
Эти данные взяты с конкурса с сайта kaggle.\
Ссылка на соревнование https://www.kaggle.com/c/sberbank-russian-housing-market/overview
    
Если интеерсно - можно попробоватаь зарегистрироваться там и попробовать заслать решения, которые получится сделать. Проверить свои силы и посоревноваться с исследователями данных.


**Конкурс**:

Цель этого конкурса - спрогнозировать продажную цену каждого объекта недвижимости. Целевая переменная называется price_doc в train.csv.

Данные обучения - с августа 2011 года по июнь 2015 года, а набор тестов - с июля 2015 года по май 2016 года. Набор данных также включает информацию об общих условиях в российской экономике и финансовом секторе, поэтому вы можете сосредоточиться на создании точных прогнозов цен для отдельных объектов недвижимости.

## Check-list:


    1) Посмотреть на метрику, которую поставили организаторы конкурса
    2) Посмотреть на данные, которые они предоставили сделать простые операции с ними
    3) Собрать отложенную выборку
    4) Предобработать данные
    5) Обучить линейную регрессию
    6) Написать функцию, которая возвращала бы определенную метрику на нашей отложеной выборке
    7) Выписать возможные гипотезы для улучшения
    8) Посчитать результат каждой гипотезы

### 1. Посмотреть на метрику, которую поставили организаторы конкурса

Метрика, которую нам нужно будет улучшать это root mean squared logarithmic error (RMSLE).
Она считается по формуле $$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n{(log(x_i + 1) - log(y_i + 1))^2}}$$

Также не могу не прикрепить статью, которая сравнивает RMSE и  RMSLE
https://medium.com/analytics-vidhya/root-mean-square-log-error-rmse-vs-rmlse-935c6cc1802a

### 2. Посмотрим на данные, которые есть

In [2]:
import pandas as pd 

In [3]:
# Загрузим данные
df = pd.read_csv('train.csv')

Посмотрим как выглядят наши данные.

In [4]:
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,30469,2015-06-30,44,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,...,15,5,0,15,26,1,2,84,6,7400000
30467,30470,2015-06-30,86,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,...,313,128,24,98,182,1,82,171,15,25000000
30468,30471,2015-06-30,45,NaN,10.0,20.0,1.0,NaN,1.0,1.0,...,1,1,0,2,12,0,1,11,1,6970959
30469,30472,2015-06-30,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,...,22,1,1,6,31,1,4,65,7,13500000


Мы видим, что есть много переменных с пропущеными значениями. В этом уроке мы заполним пропуски нулями, но на самом деле есть много стратегий, как лучше обрабатывать пропуски в данных. Об этом мы поговорим в следующих лекциях.

In [5]:
df.fillna(0, inplace=True)

Также посмотим в файл с описанием данных - выберем переменные, которые нам могли бы помочь.

**price_doc** : цена продажи (это целевая переменная)\
**id**: идентификатор транзакции\
**timestamp**: дата транзакции\
**full_sq**: общая площадь в квадратных метрах, включая лоджии, балконы и другие нежилые помещения\
**life_sq**: жилая площадь в квадратных метрах без учета лоджий, балконов и других нежилых помещений\
**floor**: для квартир, этаж дома\
**max_floor**: количество этажей в здании\
**material**: материал стены\
**build_year**: год постройки\
**num_room**: количество жилых комнат\
**kitch_sq**: кухонная зона\
**state**: состояние квартиры\
**product_type**: покупка или инвестиции собственником-арендатором\
**sub_area**: название района

Это основные признаки, с которыми будем работать в этом уроке. 

In [6]:
#Отберем только признаки, которые описаны выше
columns = ['price_doc', 'timestamp', 'full_sq', 'life_sq',
'floor', 'max_floor', 'material', 'build_year',
 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area']

df = df[columns]

In [7]:
#посмотрим на типы выбранных колонок
df.dtypes

price_doc         int64
timestamp        object
full_sq           int64
life_sq         float64
floor           float64
max_floor       float64
material        float64
build_year      float64
num_room        float64
kitch_sq        float64
state           float64
product_type     object
sub_area         object
dtype: object

Мы видим, что есть несколько признаков с типом object - скорее всего это какието категориальные признаки. Линейная регерессия с ними не очень хорошо работает, поэтому давайте пока их также удалим.

In [8]:
df = df._get_numeric_data()
df.head()

,price_doc,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state
0,5850000,43,27.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6000000,34,19.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5700000,43,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13100000,89,50.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16331452,77,77.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0


Отлично! Пропусков нет, все колонки имеют количественные значения.

### 3. Отложим часть выборки

In [9]:
#Отложим часть выборки, чтобы убедиться в дальнейшем, что наша модель не переобучилась.
df_test = df.sample(n=1000, random_state=0)
df_train = df.drop(index=df_test.index)

df_test = df_test.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

In [13]:
#Зададим объекты и целевое значение

X_train = df_train[df_train.columns[1:]]
y_train = df_train['price_doc']

X_test = df_test[df_test.columns[1:]]
y_test = df_test['price_doc']

In [14]:
# df_train.columns[1:]


### 4. Построим самый простой алгоритм посчитаем его качество


Метрика, которую обсуждали выше уже чатично реализована в sklearn, нам нужно будет только извлечь из нее корень.

In [16]:
pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 4.2 MB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 9.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
from sklearn.metrics import mean_squared_log_error

In [18]:
predictions = np.array([y_train.median()]*1000)

NameError: name 'np' is not defined

In [ ]:
print(f"Значение метрики RMSLE при предскзаании медианой: {np.sqrt(mean_squared_log_error( y_test, predictions ))}")

In [ ]:
baseline = np.sqrt(mean_squared_log_error(y_test, predictions))

Лучший скор на лидерборде 0.3, метрика чем меньше, тем лучше, значит нам есть куда стремиться.

### 5. Займемся предобработкой данных

Вспомним преобразования из лекции\
$$x_i^j = \frac{x_i^j - min(x_1^j,x_2^j, \dots, x_i^j)}{max(x_1^j,x_2^j, \dots, x_i^j) - min(x_1^j,x_2^j, \dots, x_i^j)}$$

$$x_i^j = \frac{x_i^j - \mu_j}{\sigma_j}$$

В sklearn уже есть готовые преобразователи, которые и необходимые величины посчитают и данные преобразуют.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

**Ссылки на их документации**


https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [ ]:
# зададим объекты - трансформеры, 
# которые преобразуют наши данные и запомнят средние,
# максимальные и другие необходимые для вычислений значения

min_max = MinMaxScaler() 
standard = StandardScaler()

# метод фит вычисляет все вспомогательные значения по каждому признаку
min_max.fit(X_train)
standard.fit(X_train)

# метод трансформ преобразует с помощью вычесленных значений значения признаков из нашего датасета
X_train_min_max = min_max.transform(X_train)
X_test_min_max = min_max.transform(X_test)

#нам нужно преобразовать как тестовую, так и тренировочную выборку,
#хорошим тоном считается вычислять вспомогательные значения по обучающей выборке
X_train_standard = standard.transform(X_train)
X_test_standard = standard.transform(X_test)

###  6. Обучим линейную регрессию

In [20]:
import sklearn.linear_model as lm

**Ссылка на документацию:**

Линейная регрессия https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html \
L1-регрессия (Lasso) https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso \
L2-регрессия (Ridge) https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

In [21]:
linear_regression = lm.LinearRegression(normalize=False).fit(X=X_train_min_max, y=y_train)

TypeError: __init__() got an unexpected keyword argument 'normalize'

### 7. Посчитаем качество, с которым у нас получилось предсказать

In [83]:
# Давайте запишем для этого отдельную функцию, 
# которая бы получала на вход тренировочные объекты,
# ответы на тренироввочных объектах и также тестовые объекты и ответы на тестовых объектах, а также модель
# Возвращала бы она метрикиу, которы посчиталась сделав предсказания на тренировочные  и на тестовые объекты
# X_train, y_train - тренировочные объекты и ответы
# X_test, y_test - тестовые объекты и ответы

def calculate_metric(X_train, X_test, y_train, y_test, model):

    predictions_test = model.predict(X_test)
    predictions_train = model.predict(X_train)
    
    if (predictions_test < 0).any() or (predictions_train < 0).any():
        predictions_test[np.where(predictions_test < 0)] = 0
        predictions_train[np.where(predictions_train < 0)] = 0
    

    metric_test = np.sqrt(mean_squared_log_error( y_test, predictions_test))
    metric_train = np.sqrt(mean_squared_log_error( y_train, predictions_train))

    delta_baseline = baseline - metric_test

    print(f"Значение метрики RMSLE на тесте : {metric_test}")
    print(f"Значение метрики RMSLE на трейне: {metric_train}")
    print(f"Улучшение метрики по сравнению с бейзлайонм: {delta_baseline}")

Здорово! Теперь мы можем считать то что нам нужно используя всего лишь одну функцию.

In [84]:
calculate_metric(X_train=X_train_min_max, X_test=X_test_min_max, y_train=y_train, y_test=y_test, model=linear_regression)

Значение метрики RMSLE на тесте : 0.5588000012725934
Значение метрики RMSLE на трейне: 0.5571633028324696
Улучшение метрики по сравнению с бейзлайонм: 0.045538721853122355


Отлично! Мы улучшили нашу метрику на 0.045, есть куда двигаться дальше.

### 8. Гипотезы, которые можем попробовтаь опираясь на пройденный урок.
1) Попробовать стандартизовать признаки\
2) Попробовать ввести квадраты признаков\
3) Попробовать регуляризацию для модели


Для стандартизации у нас уже записан код и посчитано в отдельном датасете X_train_standard

**ДЗ1:**
    Нужно используя уже готовое стандартное преобразование обучить линейную регрессию и посчитать какая получится метрика на тесте и округлить ее до второго знака после запятой.

In [32]:
#здесь можно написать код с обучением линейной регрессии
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression

# Загрузка данных
df = pd.read_csv('train.csv')
df.fillna(0, inplace=True)

# Выбор нужных признаков
columns = ['price_doc', 'timestamp', 'full_sq', 'life_sq',
           'floor', 'max_floor', 'material', 'build_year',
           'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area']

df = df[columns]

# Оставляем только числовые данные
df = df._get_numeric_data()

# Разделение на тренировочную и тестовую выборки
df_test = df.sample(n=1000, random_state=0)
df_train = df.drop(index=df_test.index)

df_test = df_test.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

X_train = df_train[df_train.columns[1:]]
y_train = df_train['price_doc']
X_test = df_test[df_test.columns[1:]]
y_test = df_test['price_doc']

# Стандартизация данных
standard = StandardScaler()
standard.fit(X_train)

X_train_standard = standard.transform(X_train)
X_test_standard = standard.transform(X_test)

# Обучение линейной регрессии
linear_regression = LinearRegression().fit(X=X_train_standard, y=y_train)

# Функция для расчета метрики
def calculate_metric(X_train, X_test, y_train, y_test, model):
    predictions_test = model.predict(X_test)
    predictions_train = model.predict(X_train)
    
    if (predictions_test < 0).any() or (predictions_train < 0).any():
        predictions_test[np.where(predictions_test < 0)] = 0
        predictions_train[np.where(predictions_train < 0)] = 0
    
    metric_test = np.sqrt(mean_squared_log_error(y_test, predictions_test))
    metric_train = np.sqrt(mean_squared_log_error(y_train, predictions_train))
    delta_baseline = baseline - metric_test

    print(f"Значение метрики RMSLE на тесте : {metric_test:.2f}")
    print(f"Значение метрики RMSLE на трейне: {metric_train:.2f}")
    print(f"Улучшение метрики по сравнению с бейзлайном: {delta_baseline:.2f}")

In [33]:
#здесь с помощью написанной функции можно посчитать качество которое получается при такой обработке данных
predictions = np.array([y_train.median()] * len(y_test))
baseline = np.sqrt(mean_squared_log_error(y_test, predictions))

In [34]:
#здесь можно вставить код, для округления до второго знака после запятой или посчитать устно
calculate_metric(X_train=X_train_standard, X_test=X_test_standard, y_train=y_train, y_test=y_test, model=linear_regression)

Значение метрики RMSLE на тесте : 0.56
Значение метрики RMSLE на трейне: 0.56
Улучшение метрики по сравнению с бейзлайном: 0.05


**2) генерация квадратов фичей**

Для генерации новых фичей, нам нужно работать с изначальным датасетом, давайте так и сделаем.

In [22]:
for c in df.columns[1:]:
    df[c+'_2'] = df[c]*df[c]
    df[c+'_3'] = df[c+'_2']*df[c]
    df[c+'_4'] = df[c+'_3']*df[c]

In [23]:
df

,price_doc,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,build_year_4,num_room_2,num_room_3,num_room_4,kitch_sq_2,kitch_sq_3,kitch_sq_4,state_2,state_3,state_4
0,5850000,43,27.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6000000,34,19.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5700000,43,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13100000,89,50.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16331452,77,77.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,7400000,44,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,3.0,...,1.521488e+13,4.0,8.0,16.0,36.0,216.0,1296.0,9.0,27.0,81.0
30467,25000000,86,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,3.0,...,1.401922e+13,16.0,64.0,256.0,100.0,1000.0,10000.0,9.0,27.0,81.0
30468,6970959,45,0.0,10.0,20.0,1.0,0.0,1.0,1.0,1.0,...,0.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30469,13500000,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,2.0,...,1.609622e+13,4.0,8.0,16.0,121.0,1331.0,14641.0,4.0,8.0,16.0


In [24]:
#Отложим часть выборки, чтобы убедиться в дальнейшем, что наша модель не переобучилась.
df_test = df.sample(n=1000, random_state=0)
df_train = df.drop(index=df_test.index)

df_test = df_test.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

In [25]:
X_train = df_train[df_train.columns[1:]]
y_train = df_train['price_doc']

X_test = df_test[df_test.columns[1:]]
y_test = df_test['price_doc']

In [39]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
min_max = MinMaxScaler()
standard = StandardScaler()

min_max.fit(X_train)
standard.fit(X_train)

X_train_min_max = min_max.transform(X_train)
X_test_min_max = min_max.transform(X_test)

X_train_standard = standard.transform(X_train)
X_test_standard = standard.transform(X_test)

In [40]:
linear_regression = lm.LinearRegression().fit(X=X_train_min_max, y=y_train)

In [41]:
calculate_metric(X_train=X_train_min_max, X_test=X_test_min_max, y_train=y_train, y_test=y_test, model=linear_regression)

Значение метрики RMSLE на тесте : 0.56
Значение метрики RMSLE на трейне: 0.56
Улучшение метрики по сравнению с бейзлайном: 0.05


In [42]:
ridge = lm.Ridge().fit(X=X_train_min_max, y=y_train)

In [43]:
linear_regression.coef_

array([ 1.68773583e+08,  7.23869728e+07,  9.29815188e+06, -7.22913857e+06,
       -9.01149330e+04, -2.70477942e+06,  2.47487612e+07,  2.30143797e+06,
        5.94583262e+06])

In [97]:
calculate_metric(X_train=X_train_min_max, X_test=X_test_min_max, y_train=y_train, y_test=y_test, model=ridge)

Значение метрики RMSLE на тесте : 0.5543492717965641
Значение метрики RMSLE на трейне: 0.5544899453593612
Улучшение метрики по сравнению с бейзлайонм: 0.04998945132915167


**Дз2:**
    Нужно, используя уже известный код для обучения Ridge регрессии обучить Lasso регрессию, это делается с помощью lm.Lasso().Нужно будет обучить и посчитать получившееся качество.

In [44]:
# Здесь можно вставить код для обучения линейной регрессии с регуляризацией лассо
lasso = lm.Lasso().fit(X=X_train_min_max, y=y_train)

In [45]:
# Здесь можно вставить код с расчетом качества лассо
calculate_metric(X_train=X_train_min_max, X_test=X_test_min_max, y_train=y_train, y_test=y_test, model=lasso)

Значение метрики RMSLE на тесте : 0.56
Значение метрики RMSLE на трейне: 0.56
Улучшение метрики по сравнению с бейзлайном: 0.05


**Дз3:**
    Нужно,используя обученную линейную регрессию Lasso ответить на вопрос - есть ли признаки, которые можно удалить. То есть те, перед которыми коэффициент равен нулю.

In [46]:
#Здесь можно посмотреть коэффициенты линейной регрессии Лассо и ответить на вопрос - какую фичу можно удалить, ответом будет ее название
lasso.coef_

array([ 1.68754862e+08,  7.23665433e+07,  9.29783766e+06, -7.22855605e+06,
       -9.00665617e+04, -2.67288165e+06,  2.47497494e+07,  2.29476612e+06,
        5.94329580e+06])

**Итог:** Наилучший результат удалось получить с помощью линейной регрессии, в которой не используется регуляризация. Также нам помогла генерация вспомогательных фичей - квадратов, кубов и 4-х степеней изначальных признаков.